# Function Calling

Read:

https://platform.openai.com/docs/guides/function-calling

https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models


In [1]:
from openai import OpenAI
import json
from dotenv import load_dotenv, find_dotenv

_ : bool = load_dotenv(find_dotenv()) # read local .env file

client : OpenAI = OpenAI()
print(_)
print(type(_))

True
<class 'bool'>


## The basic sequence of steps for function calling is as follows:

1. Call the model with the user query and a set of functions defined in the functions parameter.
2. The model can choose to call one or more functions; if so, the content will be a stringified JSON object adhering to your custom schema (note: the model may hallucinate parameters).
3. Parse the string into JSON in your code, and call your function with the provided arguments if they exist.
4. Call the model again by appending the function response as a new message, and let the model summarize the results back to the user.

![Alt text](first.png "function_calling")

![Alt text](second.png "function_calling")

https://www.linkedin.com/pulse/azure-openai-function-calling-tarun-sharma/

In [11]:
# # Example dummy function hard coded to return the same weather
# # In production, this could be your backend API or an external API
def get_current_weather(location:str, unit:str="fahrenheit")->str:
    """Get the current weather in a given location"""
    if "karachi" in location.lower():
        return json.dumps({"location": "Karachi", "temperature": "10", "unit": "celsius"})
    elif "islamabad" in location.lower():
        return json.dumps({"location": "Islamabad", "temperature": "72", "unit": "fahrenheit"})
    elif "lahore" in location.lower():
        return json.dumps({"location": "Lahore", "temperature": "22", "unit": "celsius"})
    else:
        return json.dumps({"location": location.lower(), "temperature": "sorry, does not provide the temprature from your given place"})


In [56]:
get_current_weather(location="karachi")

'{"location": "Karachi", "temperature": "10", "unit": "celsius"}'

In [57]:
get_current_weather(location="LAHORE")

'{"location": "Lahore", "temperature": "22", "unit": "celsius"}'

In [58]:
get_current_weather(location="TOKYO")

'{"location": "tokyo", "temperature": "sorry, does not provide the temprature from your given place"}'

### Types

https://github.com/openai/openai-python/tree/main/src/openai/types/chat


In [70]:
from openai.types.chat.chat_completion import ChatCompletionMessage, ChatCompletion
# from openai.types.chat.chat_completion import ChatCompletionMessageParam, ChatCompletionMessageParam

def run_conversation(main_request: str)->str:
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": main_request}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]

    # First Request
    response: ChatCompletion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message: ChatCompletionMessage = response.choices[0].message
    display("* First Response: ", dict(response_message))
    print("=============================")
    tool_calls = response_message.tool_calls
    display("* First Response Tool Calls: ", list(tool_calls))

    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function via FastAPI
        messages.append(response_message)  # extend conversation with assistant's reply

        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
          if tool_call.type == "function":  
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            function_response = call_fastapi_get_current_weather(
                location=function_args.get("location"),
                unit=function_args.get("unit"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": json.dumps(function_response),
                }
            )  # extend conversation with function response
        display("* Second Request Messages: ", list(messages))
        second_response: ChatCompletion = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        print("* Second Response: ", dict(second_response))
        return second_response.choices[0].message.content

# run_conversation("What's the weather like in lahore, karachi, and islamabad?")
run_conversation("What's the weather like in lahore, karachi, islamabad?")

'* First Response: '

{'content': None,
 'role': 'assistant',
 'function_call': None,
 'tool_calls': [ChatCompletionMessageToolCall(id='call_Otm8EMWDgX2lrEblWpDDRuTH', function=Function(arguments='{"location": "Lahore", "unit": "celsius"}', name='get_current_weather'), type='function'),
  ChatCompletionMessageToolCall(id='call_oRNbHAC9K96pR0pIVP3JSg98', function=Function(arguments='{"location": "Karachi", "unit": "celsius"}', name='get_current_weather'), type='function'),
  ChatCompletionMessageToolCall(id='call_eYDBmZZcDOpF98YRIGyUbXWb', function=Function(arguments='{"location": "Islamabad", "unit": "celsius"}', name='get_current_weather'), type='function')]}

'* First Response Tool Calls: '

[ChatCompletionMessageToolCall(id='call_Otm8EMWDgX2lrEblWpDDRuTH', function=Function(arguments='{"location": "Lahore", "unit": "celsius"}', name='get_current_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_oRNbHAC9K96pR0pIVP3JSg98', function=Function(arguments='{"location": "Karachi", "unit": "celsius"}', name='get_current_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_eYDBmZZcDOpF98YRIGyUbXWb', function=Function(arguments='{"location": "Islamabad", "unit": "celsius"}', name='get_current_weather'), type='function')]

'* Second Request Messages: '

[{'role': 'user',
  'content': "What's the weather like in lahore, karachi, islamabad?"},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Otm8EMWDgX2lrEblWpDDRuTH', function=Function(arguments='{"location": "Lahore", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_oRNbHAC9K96pR0pIVP3JSg98', function=Function(arguments='{"location": "Karachi", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_eYDBmZZcDOpF98YRIGyUbXWb', function=Function(arguments='{"location": "Islamabad", "unit": "celsius"}', name='get_current_weather'), type='function')]),
 {'tool_call_id': 'call_Otm8EMWDgX2lrEblWpDDRuTH',
  'role': 'tool',
  'name': 'get_current_weather',
  'content': '"{\\"location\\": \\"Lahore\\", \\"temperature\\": \\"22\\", \\"unit\\": \\"celsius\\"}"'},
 {'tool_call_id': 'call_oRNbHAC9K96pR0pIVP3JSg98',


* Second Response:  {'id': 'chatcmpl-9XVWWGZquEvO2UU66tVpeA6cn0JwA', 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Currently, the weather in Lahore is 22°C, in Karachi is 10°C, and in Islamabad is 22.2°C.', role='assistant', function_call=None, tool_calls=None))], 'created': 1717773028, 'model': 'gpt-3.5-turbo-1106', 'object': 'chat.completion', 'system_fingerprint': 'fp_482d920018', 'usage': CompletionUsage(completion_tokens=28, prompt_tokens=178, total_tokens=206)}


'Currently, the weather in Lahore is 22°C, in Karachi is 10°C, and in Islamabad is 22.2°C.'

In [71]:
from openai.types.chat.chat_completion import ChatCompletionMessage, ChatCompletion
# from openai.types.chat.chat_completion import ChatCompletionMessageParam, ChatCompletionMessageParam

def run_conversation(main_request: str)->str:
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": main_request}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]

    # First Request
    response: ChatCompletion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message: ChatCompletionMessage = response.choices[0].message
    display("* First Response: ", dict(response_message))
    print("=============================")
    tool_calls = response_message.tool_calls
    display("* First Response Tool Calls: ", list(tool_calls))

    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function via FastAPI
        messages.append(response_message)  # extend conversation with assistant's reply

        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
          if tool_call.type == "function":  
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            function_response = call_fastapi_get_current_weather(
                location=function_args.get("location"),
                unit=function_args.get("unit"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": json.dumps(function_response),
                }
            )  # extend conversation with function response
        display("* Second Request Messages: ", list(messages))
        second_response: ChatCompletion = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        print("* Second Response: ", dict(second_response))
        return second_response.choices[0].message.content

# run_conversation("What's the weather like in lahore, karachi, and islamabad?")
run_conversation("What's the weather like in tokyo?")

'* First Response: '

{'content': None,
 'role': 'assistant',
 'function_call': None,
 'tool_calls': [ChatCompletionMessageToolCall(id='call_ocZSvWkq7h5nHmzsECXkR1v8', function=Function(arguments='{"location":"Tokyo","unit":"celsius"}', name='get_current_weather'), type='function')]}

'* First Response Tool Calls: '

[ChatCompletionMessageToolCall(id='call_ocZSvWkq7h5nHmzsECXkR1v8', function=Function(arguments='{"location":"Tokyo","unit":"celsius"}', name='get_current_weather'), type='function')]

'* Second Request Messages: '

[{'role': 'user', 'content': "What's the weather like in tokyo?"},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ocZSvWkq7h5nHmzsECXkR1v8', function=Function(arguments='{"location":"Tokyo","unit":"celsius"}', name='get_current_weather'), type='function')]),
 {'tool_call_id': 'call_ocZSvWkq7h5nHmzsECXkR1v8',
  'role': 'tool',
  'name': 'get_current_weather',
  'content': '"{\\"location\\": \\"tokyo\\", \\"temperature\\": \\"sorry, does not provide the temperature from your given place\\"}"'}]

* Second Response:  {'id': 'chatcmpl-9XVXCX7dznSwQ8nshVBt7wo6pFn90', 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I'm sorry, but I'm unable to provide the current weather in Tokyo. You may want to check a reliable weather website or app for the most accurate and up-to-date information.", role='assistant', function_call=None, tool_calls=None))], 'created': 1717773070, 'model': 'gpt-3.5-turbo-1106', 'object': 'chat.completion', 'system_fingerprint': 'fp_b953e4de39', 'usage': CompletionUsage(completion_tokens=37, prompt_tokens=72, total_tokens=109)}


"I'm sorry, but I'm unable to provide the current weather in Tokyo. You may want to check a reliable weather website or app for the most accurate and up-to-date information."